In [1]:
import numpy as np
import pandas as pd
import xgboost 
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.svm import OneClassSVM
from sklearn.neural_network import MLPClassifier  # Multi-layer Perceptron classifier.
from sklearn.preprocessing import StandardScaler  # Scaler

/home/sirorezka/anaconda2/envs/py27/lib/python2.7/site-packages/sklearn/cross_validation.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
sklearn.neural_network??

Object `sklearn.neural_network` not found.


In [3]:
df_train = pd.read_csv("Data/train.csv")
df_test = pd.read_csv("Data/test.csv")

print(df_test.shape)
print(df_train.shape)

(75818, 370)
(76020, 371)


In [4]:
# remove constant columns
remove = []
for col in df_train.columns:
    if df_train[col].std() == 0:
        remove.append(col)

df_train.drop(remove, axis=1, inplace=True)
df_test.drop(remove, axis=1, inplace=True)

print(df_test.shape)
print(df_train.shape)

(75818, 336)
(76020, 337)


In [5]:
df_train.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,39205.170000,0
1,3,2,34,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,49278.030000,0
2,4,2,23,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,67333.770000,0
3,8,2,37,0,195,195,0,0,0,0,...,0,0,0,0,0,0,0,0,64007.970000,0
4,10,2,39,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,117310.979016,0


In [6]:
# remove duplicated columns
remove = []
c = df_train.columns
for i in range(len(c)-1):
    v = df_train[c[i]].values
    for j in range(i+1,len(c)):
        if np.array_equal(v,df_train[c[j]].values):
            remove.append(c[j])

df_train.drop(remove, axis=1, inplace=True)
df_test.drop(remove, axis=1, inplace=True)

y_train = df_train['TARGET'].values
X_train = df_train.drop(['ID','TARGET'], axis=1).values

id_test = df_test['ID']
X_test = df_test.drop(['ID'], axis=1).values

print(df_test.shape)
print(df_train.shape)

(75818, 307)
(76020, 308)


In [7]:
len_train = len(X_train)
len_test  = len(X_test)

# Scale data
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

# apply same transformation to test data
X_test = scaler.transform(X_test) 

In [ ]:
X_fit, X_eval, y_fit, y_eval = train_test_split(X_train, y_train, test_size=0.5, random_state = 84)

k_vars = [1, 5,7,9,11,13,15,17,19,21,23,25,27,29,31,33,35,50,70,85,100,120,140,160,180,200,220,250,270,285,300,315,330,350,375,400,425,450,475,500,525,550,575,600]
scores_list = []
for k1_val in k_vars:
    for k2_val in k_vars:
        for k3_val in k_vars:
            clf = MLPClassifier(hidden_layer_sizes=(k1_val,k2_val,k3_val), activation='relu', 
                 beta_1=0.9, beta_2=0.999,
                 learning_rate_init=0.001, max_iter = 5000, random_state = 1235, 
                 learning_rate='adaptive',
                 early_stopping=True, validation_fraction=0.5)

            clf.fit(X_train, y_train)
            y_pred= clf.predict_proba(X_test)[:,1]
            rc2 = roc_auc_score(y_fit, clf.predict_proba(X_fit)[:,1])
            rc1 = roc_auc_score(y_train, clf.predict_proba(X_train)[:,1])
            rc3 = roc_auc_score(y_eval, clf.predict_proba(X_eval)[:,1])
            ##print(k_val,' -- Overall AUC all,fit,eval:',rc1,rc2,rc3 )
            scores_list.append([k1_val,k2_val,k3_val,rc1,rc2,rc3])
    print (k1_val)


submission = pd.DataFrame({"ID":id_test, "TARGET":y_pred})
submission.to_csv("predictions/submission_nn.csv", index=False)

print('Completed!')

In [57]:
sorted(scores_list, key = lambda x: x[5], reverse=True)[0:35]

[[0.82612568614850257, 0.82032502770961679, 0.83177169777794435],
 [0.81741195942287537, 0.81102166123725206, 0.82363579337334691],
 [0.81370862801189203, 0.80591989023769917, 0.82127780410181439],
 [0.81226907395578218, 0.80565458254482403, 0.81869021322577407],
 [0.81389988327844109, 0.80928817777792894, 0.81838780542097811],
 [0.80914407987509496, 0.80215463931989395, 0.81590511578618108],
 [0.80859288942301522, 0.80237938282883425, 0.81458091711794944],
 [0.8079706922630614, 0.80163751760207391, 0.81413396495225121],
 [0.80845147932181938, 0.80265487546832492, 0.81408067067003831],
 [0.80765803872519937, 0.80220334591577824, 0.81293061631938413],
 [0.80409721248824151, 0.79608907846732413, 0.81187937807636945],
 [0.80197196070800369, 0.79275670626515993, 0.81090577464998315],
 [0.80302786590167052, 0.79646568119390648, 0.80939921942634574],
 [0.80174089123428849, 0.79482366816711947, 0.80839980354532093],
 [0.80181086206245888, 0.79556780698450358, 0.80783786209977926],
 [0.7994828

In [50]:
sorted??